## Analyze elongation time and collision frequencies

In [7]:
import cPickle as pkl
import pandas as pd
import numpy as np

In [8]:
files = [
"/home/martin/git/TRSL/results/1000 ribosomes, 30061 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180405_0023_1000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/2000 ribosomes, 60000 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180405_0911_2000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/1000 ribosomes, 60000 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180404_2001_1000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/2000 ribosomes, 30061 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180405_1345_2000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/5000 ribosomes, 60000 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180406_0032_5000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/5000 ribosomes, 30061 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180406_0627_5000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/10000 ribosomes, 60000 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180406_1659_10000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/10000 ribosomes, 30061 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180406_2111_10000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/20000 ribosomes, 60000 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180407_0548_20000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/20000 ribosomes, 30061 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180407_1116_20000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/200000 ribosomes, 60000 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180410_1948_200000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/50000 ribosomes, 60000 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180407_2350_50000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/50000 ribosomes, 30061 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180408_0821_50000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/100000 ribosomes, 60000 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180409_0259_100000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/200000 ribosomes, 30061 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180411_1552_200000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/100000 ribosomes, 30061 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180409_1516_100000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/500000 ribosomes, 60000 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180413_1955_500000_ribosomes_1200s.p",
"/home/martin/git/TRSL/results/500000 ribosomes, 30061 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180414_1659_500000_ribosomes_1200s.p",
]
descriptor = '_constant_tRNAs'

To convert from numpypy to numpy:

In [ ]:
from tempfile import mkstemp
from shutil import move
from os import remove, close

def replace(file_path, pattern, subst):
    #Create temp file
    fh, abs_path = mkstemp()
    with open(abs_path,'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                new_file.write(line.replace(pattern, subst))
    close(fh)
    #Remove original file
    remove(file_path)
    #Move new file
    move(abs_path, file_path)

Run only once per file:

In [ ]:
for filename in files:
    print filename
    replace(filename, "_numpypy.multiarray", "numpy.core.multiarray")

/home/martin/git/TRSL/results/1000 ribosomes, 30061 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180405_0023_1000_ribosomes_1200s.p
/home/martin/git/TRSL/results/2000 ribosomes, 60000 phase transcriptome, constant tRNAs, full exome, median-enhanced initiation rates according to Shah, deltat=0.05s_20180405_0911_2000_ribosomes_1200s.p


Blacklisted species:

In [ ]:
blacklist = ['ribos._bound', 'ribos._free', 'tRNA_free', 'tRNA_bound', 'ATP', 'GTP', 'AMP', 'GDP', 'protein', 
             'proteins', 'peptide_bonds']

Data frame at cell level:

In [ ]:
dfc = pd.DataFrame(columns=['ribosomes', 'transcripts', 'collisions', 'nocollisions', 'peptide_bonds'])

Fill data frames:

In [ ]:
rows = []
dft_columns = ['length', 'init_rate', 'ribosomes', 'tic_toc', 'geneID', 'transcripts', 'index']

for i, filename in enumerate(files):
    print filename
    results = pkl.load(open(filename))
    
    # append to cell level data frame
    print 'cell level...'
    rowvals = [results['n_ribosomes'], len(results['transcriptome']), results['collisions'], 
               results['nocollisions'], results['peptide_bonds']]
    row = dict(zip(list(dfc.columns), rowvals))
    #print row
    dfc = dfc.append(row, ignore_index=True)
    #print dfc.head()
    
    # append to transcript level data frame
    print 'transcript level...'
    
    for transcript in results['transcriptome']:
        av_time = np.mean([tictoc[1] - tictoc[0] for tictoc in transcript.tic_toc])
        rowvals = [transcript.geneID, transcript.index, transcript.init_rate, transcript.length, 
                   results['n_ribosomes'], av_time, len(results['transcriptome'])]
        row = dict(zip(list(dft_columns), rowvals))
        #print row
        rows.append(row)
            
    # append to gene level data frame
    print 'gene level...'
    header = str(results['n_ribosomes']) + '_' + str(len(results['transcriptome']))
    tc = results['timecourses']
    efficiency_items = [(gene, (tc[gene][-1] - tc[gene][-6001]) / 300.) for gene in tc if gene not in blacklist]
    tmp = pd.DataFrame(efficiency_items, columns = ['gene', header]).set_index('gene')
    
    if i == 0:  # first time
        dfg = tmp
    else:
        dfg = pd.merge(dfg, tmp, left_index=True, right_index=True, how='outer')

dft = pd.DataFrame(rows)
dft.columns = dft_columns

In [ ]:
dfc

In [ ]:
dft.columns = ['ribosomes', 'transcripts', 'index', 'geneID', 'init_rate', 'length', 'tic_toc']

In [ ]:
dft

In [ ]:
dfg

In [ ]:
dfc.to_pickle('../../results/dfc' + descriptor + '.p')

In [ ]:
dft.to_pickle('../../results/dft' + descriptor + '.p')

In [ ]:
dfg.to_pickle('../../results/dfg' + descriptor + '.p')